In [1]:
import sys
print(sys.version)
username = 'admin' # 'admin' for BO1 lab
sys.path.append('c:/users/' + username + '/picosdk-python-wrappers')
import picosdk
import os
os.getcwd()
import logging
import ctypes
import numpy as np #
from picosdk.ps4000 import ps4000 as ps #
import matplotlib.pyplot as plt
from picosdk.functions import adc2mV, assert_pico_ok
from statistics import mean
import time
import math

%matplotlib inline

def round_half_up(n, decimals=0):
    multiplier = 10 ** decimals
    return math.floor(n*multiplier + 0.5) / multiplier

3.7.3 (default, Apr 24 2019, 15:29:51) [MSC v.1915 64 bit (AMD64)]


CannotOpenPicoSDKError: PicoSDK (ps4000) not compatible (check 32 vs 64-bit): [WinError 193] %1 is not a valid Win32 application

In [ ]:
#Configure the logging
logging.basicConfig(level=logging.INFO)
logging.getLogger().setLevel(logging.INFO)

def streaming_callback(handle, noOfSamples, startIndex, overflow, triggerAt, triggered, autoStop, param):
    global nextSample, autoStopOuter, wasCalledBack
    wasCalledBack = True
    destEnd = nextSample + noOfSamples
    sourceEnd = startIndex + noOfSamples
    bufferCompleteA[nextSample:destEnd] = bufferAMax[startIndex:sourceEnd]
    bufferCompleteB[nextSample:destEnd] = bufferBMax[startIndex:sourceEnd]
    nextSample += noOfSamples
    if autoStop:
        autoStopOuter = True

def pico_start():
    # Create chandle and status ready for use
    chandle = ctypes.c_int16()
    status = {}

    # Open PicoScope 2000 Series device
    # Returns handle to chandle for use in future API functions
    status["openunit"] = ps.ps4000OpenUnit(ctypes.byref(chandle))
    assert_pico_ok(status["openunit"])


    enabled = 1
    disabled = 0
    analogue_offset = 0.0
    nextSample = 0
    autoStopOuter = False
    wasCalledBack = False

    # Set up channel A
    # handle = chandle
    # channel = PS4000_CHANNEL_A = 0
    # enabled = 1
    # coupling type = PS4000_DC = 1
    # range = PS4000_2V = 7
    channel_range = ps.PS4000_RANGE['PS4000_2V']
    status["setChA"] = ps.ps4000SetChannel(chandle,
                                            ps.PS4000_CHANNEL['PS4000_CHANNEL_A'],
                                            enabled,
                                            1,
                                            channel_range)
    assert_pico_ok(status["setChA"])

    # Set up channel B
    # handle = chandle
    # channel = PS4000_CHANNEL_B = 1
    # enabled = 1
    # coupling type = PS4000_DC = 1
    # range = PS4000_2V = 7
    status["setChB"] = ps.ps4000SetChannel(chandle,
                                            ps.PS4000_CHANNEL['PS4000_CHANNEL_B'],
                                            enabled,
                                            1,
                                            channel_range)
    assert_pico_ok(status["setChB"])

    # Size of capture
    sizeOfOneBuffer = 500
    numBuffersToCapture = 10

    totalSamples = sizeOfOneBuffer * numBuffersToCapture

    # Create buffers ready for assigning pointers for data collection
    bufferAMax = np.zeros(shape=sizeOfOneBuffer, dtype=np.int16)
    bufferBMax = np.zeros(shape=sizeOfOneBuffer, dtype=np.int16)
    # We need a big buffer, not registered with the driver, to keep our complete capture in.
    bufferCompleteA = np.zeros(shape=totalSamples, dtype=np.int16)
    bufferCompleteB = np.zeros(shape=totalSamples, dtype=np.int16)

    memory_segment = 0



    # Set data buffer location for data collection from channel A
    # handle = chandle
    # source = PS4000_CHANNEL_A = 0
    # pointer to buffer max = ctypes.byref(bufferAMax)
    # pointer to buffer min = ctypes.byref(bufferAMin)
    # buffer length = maxSamples
    # segment index = 0
    # ratio mode = PS4000_RATIO_MODE_NONE = 0
    status["setDataBuffersA"] = ps.ps4000SetDataBuffers(chandle,
                                                         ps.PS4000_CHANNEL['PS4000_CHANNEL_A'],
                                                         bufferAMax.ctypes.data_as(ctypes.POINTER(ctypes.c_int16)),
                                                         None,
                                                         sizeOfOneBuffer)
    assert_pico_ok(status["setDataBuffersA"])

    # Set data buffer location for data collection from channel B
    # handle = chandle
    # source = PS4000_CHANNEL_B = 1
    # pointer to buffer max = ctypes.byref(bufferBMax)
    # pointer to buffer min = ctypes.byref(bufferBMin)
    # buffer length = maxSamples
    # segment index = 0
    # ratio mode = PS4000_RATIO_MODE_NONE = 0
    status["setDataBuffersB"] = ps.ps4000SetDataBuffers(chandle,
                                                         ps.PS4000_CHANNEL['PS4000_CHANNEL_B'],
                                                         bufferBMax.ctypes.data_as(ctypes.POINTER(ctypes.c_int16)),
                                                         None,
                                                         sizeOfOneBuffer)
    assert_pico_ok(status["setDataBuffersB"])

    # Begin streaming mode:
    sampleInterval = ctypes.c_int32(250)
    sampleUnits = ps.PS4000_TIME_UNITS['PS4000_US']
    # We are not triggering:
    maxPreTriggerSamples = 0
    autoStopOn = 1
    # No downsampling:
    downsampleRatio = 1

    actualSampleInterval = sampleInterval.value
    actualSampleIntervalNs = actualSampleInterval * 1000
    totalSamplingTime = totalSamples * actualSampleIntervalNs
    logging.info("Capturing at sample interval %10.3E ns, with total sampling time of %10.3E ns" % (actualSampleIntervalNs, totalSamplingTime))

def pico_acquire_measurement(discarded_portion = 0.0, plot = False):    
    global status
    
    status["runStreaming"] = ps.ps4000RunStreaming(chandle,
                                                ctypes.byref(sampleInterval),
                                                sampleUnits,
                                                maxPreTriggerSamples,
                                                totalSamples,
                                                autoStopOn,
                                                downsampleRatio,
                                                sizeOfOneBuffer)
    assert_pico_ok(status["runStreaming"])
    
    # We need a big buffer, not registered with the driver, to keep our complete capture in.
    global bufferCompleteA 
    global bufferCompleteB 
    global nextSample 
    global autoStopOuter 
    global wasCalledBack 
    global cFuncPtr 
    
    bufferCompleteA = np.zeros(shape=totalSamples, dtype=np.int16)
    bufferCompleteB = np.zeros(shape=totalSamples, dtype=np.int16)
    nextSample = 0
    autoStopOuter = False
    wasCalledBack = False
    cFuncPtr = ps.StreamingReadyType(streaming_callback)
    
    while nextSample < totalSamples and not autoStopOuter:
        wasCalledBack = False
        status["getStreamingLastestValues"] = ps.ps4000GetStreamingLatestValues(chandle, cFuncPtr, None)
        if not wasCalledBack:
            # If we weren't called back by the driver, this means no data is ready. Sleep for a short while before trying
            # again.
            time.sleep(0.01)
    
    logging.info("Done grabbing values.")
    
    # Find maximum ADC count value
    # handle = chandle
    # pointer to value = ctypes.byref(maxADC)
    maxADC = ctypes.c_int16(32767)

    # Convert ADC counts data to mV
    adc2mVChAMax = adc2mV(bufferCompleteA, channel_range, maxADC)
    #adc2mVChBMax = adc2mV(bufferCompleteB, channel_range, maxADC)
    
    # Stop the scope
    # handle = chandle
    status["stop"] = ps.ps4000Stop(chandle)
    assert_pico_ok(status["stop"])
    
    if plot:
        # Plot data from channel A
        # Create time data
        time_axis = np.linspace(0, (totalSamples) * actualSampleIntervalNs, totalSamples)
        plt.plot(time_axis, adc2mVChAMax[:])
        #plt.plot(time, adc2mVChBMax[:])
        plt.xlabel('Time (ns)')
        plt.ylabel('Voltage (mV)')
        plt.show()
    print(len(adc2mVChAMax))
    print(len(adc2mVChAMax[math.floor((len(adc2mVChAMax)-1)*discarded_portion):]))
    return round_half_up(mean(adc2mVChAMax[math.floor((len(adc2mVChAMax)-1)*discarded_portion):]),3); #round_half_up(mean(adc2mVChAMax),3);

def pico_stop():
    global status
    
    # Stop the scope
    # handle = chandle
    status["stop"] = ps.ps4000Stop(chandle)
    assert_pico_ok(status["stop"])    

    # Disconnect the scope
    # handle = chandle
    status["close"] = ps.ps4000CloseUnit(chandle)
    assert_pico_ok(status["close"])
    return;

In [ ]:
pico_start()

# We need a big buffer, not registered with the driver, to keep our complete capture in.
for i in range(4):

    print(pico_acquire_measurement());
    time.sleep(0.1)

pico_stop()

# Display status returns
logging.info(status)